In [1]:
%matplotlib inline

from beep import structure
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Load y from Xiao's formation data

In [2]:
csvFile = pd.read_csv(os.getcwd() + "/tsfresh_feature_extraction/one_time_features_110323.csv")

In [3]:
y = csvFile.regu_life

In [4]:
ind_y = np.array([x for x in range(np.size(csvFile,0))])
ind_y = ind_y[~np.isnan(y)]
seq_num = csvFile.seq_num[ind_y]
y = y[ind_y]

ind_y = ind_y[seq_num != 206] # error loading Nova_Formation-206.csv
seq_num = np.array(seq_num[ind_y]) # error loading Nova_Formation-206.csv
y = np.array(y[ind_y])

In [5]:
seq_num

array([100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 112, 113,
       114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126,
       127, 128, 129, 130, 131, 134, 135, 136, 137, 138, 139, 140, 141,
       142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182,
       183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195,
       196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 207, 208, 209,
       210, 211, 212, 213, 214, 215, 216, 217, 219, 220, 221, 222, 223,
       224, 225, 226, 227, 228, 229, 230, 269, 270, 271, 272, 273, 274,
       275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287,
       288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 299, 300, 301,
       302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314,
       315, 316, 317, 318, 319, 320, 321, 322, 323], dtype=int64

In [6]:
formation_protocol_params = pd.read_csv(os.getcwd() + "/tsfresh_feature_extraction/Formation_2022_Parameter.csv")

# Extract data for steps A, B, and C

In [ ]:
A_Q_V_all = np.empty((len(ind_y), 1000))
A_t_V_all = np.empty((len(ind_y), 1000))
B_Q_V_all = np.empty((len(ind_y), 1000))
B_V_t_all = np.empty((len(ind_y), 1000))
C_Q_V_all = np.empty((len(ind_y), 1000))
C_V_t_all = np.empty((len(ind_y), 1000))

protocol_params = np.empty((len(ind_y), 6))

for ind in range(len(ind_y)):
    
    filepath = os.getcwd() + "\\data_03032025\\Formation\\Nova_Formation-" + str(seq_num[ind]) + ".csv"
    df = structure.NovonixDatapath.from_file(filepath)
    df_f = df.raw_data

    if seq_num[ind] in formation_protocol_params.seq_num.array:
        CC1 = formation_protocol_params[formation_protocol_params.seq_num==seq_num[ind]].formation_charge_current_1.array[0]
        CC2 = formation_protocol_params[formation_protocol_params.seq_num==seq_num[ind]].formation_charge_current_2.array[0]
        CV = formation_protocol_params[formation_protocol_params.seq_num==seq_num[ind]].formation_cutoff_voltage_1.array[0]
        n = formation_protocol_params[formation_protocol_params.seq_num==seq_num[ind]].formation_verification_repeat.array[0]
        T = formation_protocol_params[formation_protocol_params.seq_num==seq_num[ind]].formation_temperature.array[0]
        t_OCV = formation_protocol_params[formation_protocol_params.seq_num==seq_num[ind]].ocv_time.array[0]
    
    protocol_params[ind, :] = [CC1, CC2, CV, n, T, t_OCV]

    index_A = [(df_f.step_type_name[i] in ['cc_charge', 'cccv_charge', 'cv_hold_charge']) * (df_f.step_num[i] >= 3) * (df_f.voltage[i] >= 1.5) * (df_f.cycle_index[i] == 1) for i in range(df_f.shape[0])]
    t0 = min(df_f.test_time[index_A])
    
    V = df_f.voltage[index_A]
    Q = df_f.capacity[index_A]
    I = df_f.current[index_A]
    t = df_f.test_time[index_A]
    V_new = np.linspace(1.5, 4.4, 1000)
    A_Q_V = np.interp(V_new, V, Q)
    A_t_V = np.interp(V_new, V, t) - t0

    A_Q_V_all[ind, :] = A_Q_V
    A_t_V_all[ind, :] = A_t_V
    
    
    fig, axs = plt.subplots(figsize=(30,5), nrows=1, ncols=6)

    axs[0].plot(V_new, A_Q_V)
    axs[0].minorticks_on()
    axs[0].grid(which='major', linestyle='-', linewidth='0.5')
    axs[0].grid(which='minor', linestyle=':', linewidth='0.5')
    axs[0].set_xlabel('V_new')
    axs[0].set_ylabel('Q_V')
    axs[0].set_title('interpolated Q and V (Step A)')

    axs[1].plot(V_new, A_t_V)
    axs[1].minorticks_on()
    axs[1].grid(which='major', linestyle='-', linewidth='0.5')
    axs[1].grid(which='minor', linestyle=':', linewidth='0.5')
    axs[1].set_xlabel('V_new')
    axs[1].set_ylabel('t_V')
    axs[1].set_title('interpolated t and V (Step A)')

    index_B = [(df_f.step_type_name[i] in ['cc_discharge', 'cv_hold_discharge']) * (df_f.cycle_index[i] == n + 1) for i in range(df_f.shape[0])]
    t0 = min(df_f.test_time[index_B])
    Q0 = max(df_f.capacity[index_B])
    
    V = df_f.voltage[index_B]
    Q = Q0 - df_f.capacity[index_B]
    I = df_f.current[index_B]
    t = df_f.test_time[index_B]
    V_new = np.linspace(3.0, 4.4, 1000)
    B_Q_V = np.interp(V_new, V[::-1], Q[::-1])
    V_new = V_new[::-1]
    B_Q_V = B_Q_V[::-1]
    t_new = np.linspace(min(t), max(t), 1000) - t0
    B_V_t = np.interp(t_new, t - t0, V)

    B_Q_V_all[ind, :] = B_Q_V
    B_V_t_all[ind, :] = B_V_t

    axs[2].plot(V_new, B_Q_V)
    axs[2].minorticks_on()
    axs[2].grid(which='major', linestyle='-', linewidth='0.5')
    axs[2].grid(which='minor', linestyle=':', linewidth='0.5')
    axs[2].set_xlabel('V_new')
    axs[2].set_ylabel('Q_V')
    axs[2].set_title('interpolated Q and V (Step B)')

    axs[3].plot(t_new, B_V_t)
    axs[3].minorticks_on()
    axs[3].grid(which='major', linestyle='-', linewidth='0.5')
    axs[3].grid(which='minor', linestyle=':', linewidth='0.5')
    axs[3].set_xlabel('t_new')
    axs[3].set_ylabel('V_t')
    axs[3].set_title('interpolated V and t (Step B)')

    index_C = [(df_f.step_type_name[i] in ['cc_discharge', 'cv_hold_discharge']) * (df_f.cycle_index[i] == 1) for i in range(df_f.shape[0])]
    t0 = min(df_f.test_time[index_C])
    Q0 = max(df_f.capacity[index_C])
    
    V = df_f.voltage[index_C]
    Q = Q0 - df_f.capacity[index_C]
    I = df_f.current[index_C]
    t = df_f.test_time[index_C]
    V_new = np.linspace(3.0, 4.4, 1000)
    C_Q_V = np.interp(V_new, V[::-1], Q[::-1])
    V_new = V_new[::-1]
    C_Q_V = C_Q_V[::-1]
    t_new = np.linspace(min(t), max(t), 1000) - t0
    C_V_t = np.interp(t_new, t - t0, V)

    C_Q_V_all[ind, :] = C_Q_V
    C_V_t_all[ind, :] = C_V_t

    axs[4].plot(V_new, C_Q_V)
    axs[4].minorticks_on()
    axs[4].grid(which='major', linestyle='-', linewidth='0.5')
    axs[4].grid(which='minor', linestyle=':', linewidth='0.5')
    axs[4].set_xlabel('V_new')
    axs[4].set_ylabel('Q_V')
    axs[4].set_title('interpolated Q and V (Step C)')

    axs[5].plot(t_new, C_V_t)
    axs[5].minorticks_on()
    axs[5].grid(which='major', linestyle='-', linewidth='0.5')
    axs[5].grid(which='minor', linestyle=':', linewidth='0.5')
    axs[5].set_xlabel('t_new')
    axs[5].set_ylabel('V_t')
    axs[5].set_title('interpolated V and t (Step C)')
    
    plt.show()

In [8]:
import pandas as pd

protocol = pd.read_excel(os.getcwd() + "/tsfresh_feature_extraction/protocol_label.xlsx", header=0)
protocol = protocol.iloc[:,1:]
y = pd.DataFrame({'y': y})

df_A_Q_V_all = pd.concat([pd.DataFrame(A_Q_V_all), y, protocol], axis=1)
df_A_t_V_all = pd.concat([pd.DataFrame(A_t_V_all), y, protocol], axis=1)
df_B_Q_V_all = pd.concat([pd.DataFrame(B_Q_V_all), y, protocol], axis=1)
df_B_V_t_all = pd.concat([pd.DataFrame(B_V_t_all), y, protocol], axis=1)
df_C_Q_V_all = pd.concat([pd.DataFrame(C_Q_V_all), y, protocol], axis=1)
df_C_V_t_all = pd.concat([pd.DataFrame(C_V_t_all), y, protocol], axis=1)

if not os.path.exists(os.getcwd() + "/data_formation"):
    os.mkdir(os.getcwd() + "/data_formation")

df_A_Q_V_all.to_excel("data_formation/A_Q_V_all.xlsx")
df_A_t_V_all.to_excel("data_formation/A_t_V_all.xlsx")
df_B_Q_V_all.to_excel("data_formation/B_Q_V_all.xlsx")
df_B_V_t_all.to_excel("data_formation/B_V_t_all.xlsx")
df_C_Q_V_all.to_excel("data_formation/C_Q_V_all.xlsx")
df_C_V_t_all.to_excel("data_formation/C_V_t_all.xlsx")